#### Riley's new gene project has shown that when a GFP gene is induced in the e.coli, there is a consistent shift of proteome fraction of essential proteins. i.e. most essential proteins have like 10% smaller fraction compared to wild type e.coli. The purpose of this notebook is to identify any connection between the decrease in expression of these essential proteins and potential metabolic defect. For this purpose, we will 1). import the complex ids of essential proteins that have decreased proteome fraction, 2). identify the kinetic reactions catalyzed by some of these complexes, 3). perform FBA to see if there is any metabolic defect when these reactions are knocked downed (80% for instance).

#### Since nothing was observed when knocking down by target, now we will try forcing the flux through these reactions to be at most 80% of WT fluxes via constraints.

#### This notebook would focus on the variant 16 data, choosing a more meaningful percentage than 80 % and choosing a more comprehensive set of reactions to constrain.

In [1]:
import pandas as pd
import os, dill
import numpy as np
import cvxpy as cp
from typing import Iterable, Optional, Mapping, cast
from plotly.graph_objects import Scatter, Figure

from ecoli.processes.metabolism_redux_classic import NetworkFlowModel, FlowResult
os.chdir(os.path.expanduser('~/dev/vEcoli'))

%load_ext autoreload

# Data Processing

In [2]:
# Load in the data
below_line_monomer_data = pd.read_csv(
    'notebooks/Heena notebooks/Riley New '
    'Genes/proteome_fraction_scatterplot_below_line_data_16.tsv', sep='\t')
below_line_essential_monomer_data = pd.read_csv(
    'notebooks/Heena notebooks/Riley New Genes/proteome_fraction_scatterplot_below_line_essential_data_16.tsv', sep='\t')
below_line_complex_data = pd.read_csv(
    'notebooks/Heena notebooks/Riley New Genes/proteome_fraction_scatterplot_below_line_complex_data_16.tsv', sep='\t')
below_line_essential_complex_data = pd.read_csv(
    'notebooks/Heena notebooks/Riley New Genes/proteome_fraction_scatterplot_below_line_essential_complex_data_16.tsv', sep='\t')


In [3]:
below_line_monomer_data.columns

Index(['monomer_id', 'var_0_normalized_monomer_count',
       'var_of_interest_normalized_monomer_count'],
      dtype='object')

In [4]:
below_line_monomer_data['normalized_monomer_count_ratio'] = (
    below_line_monomer_data['var_of_interest_normalized_monomer_count'] /
    below_line_monomer_data['var_0_normalized_monomer_count'])
below_line_essential_monomer_data['normalized_monomer_count_ratio'] = (
    below_line_essential_monomer_data['var_of_interest_normalized_monomer_count'] /
    below_line_essential_monomer_data['var_0_normalized_monomer_count'])
below_line_complex_data['normalized_monomer_count_ratio'] = (
    below_line_complex_data['var_of_interest_normalized_monomer_count'] /
    below_line_complex_data['var_0_normalized_monomer_count'])
below_line_essential_complex_data['normalized_monomer_count_ratio'] = (
    below_line_essential_complex_data
    ['var_of_interest_normalized_monomer_count'] /
    below_line_essential_complex_data['var_0_normalized_monomer_count'])

below_line_monomer_ids = np.unique(below_line_monomer_data['monomer_id']
                                   .tolist())
below_line_essential_monomer_ids = (
    np.unique(below_line_essential_monomer_data['monomer_id'].tolist()))
below_line_complex_ids = np.unique(below_line_complex_data['complex_id']
                                   .tolist())
below_line_essential_complex_ids = (
    np.unique(below_line_essential_complex_data['complex_id'].tolist()))

# TODO: remember that the same complex could be associated with multiple
#  monomers - how do we pick the number to use for that complex? Ideally it
#  would be the limiting monomer, maybe we average?


In [5]:
below_line_monomer_data.shape[0]

3637

In [6]:
below_line_essential_monomer_data.shape[0]

371

In [7]:
# Plot the distribution for below line normalized count ratios
import plotly.express as px
fig = px.histogram(
    below_line_essential_complex_data,
    x='normalized_monomer_count_ratio',
    nbins=50,
    title='Distribution of normalized monomer count ratios for monomers below '
          'the line'
)
fig.add_vline(
    x=below_line_essential_complex_data['normalized_monomer_count_ratio']
    .mean(),
    line_dash="dot",
    line_color="red"
)
fig.show()

In [8]:
# same thing but for essential monomer
fig = px.histogram(
    below_line_essential_monomer_data,
    x='normalized_monomer_count_ratio',
    nbins=50,
    title='Distribution of normalized monomer count ratios for essential '
          'monomers below the line'
)
fig.add_vline(
    x=below_line_essential_monomer_data['normalized_monomer_count_ratio']
    .mean(),
    line_dash="dot",
    line_color="red"
)
fig.show()

In [9]:
# Same thing but for monomers associated with complexes
fig = px.histogram(
    below_line_complex_data,
    x='normalized_monomer_count_ratio',
    nbins=50,
    title='Distribution of normalized monomer count ratios for monomers below the line '
          'associated with complexes'
)
fig.add_vline(
    x=below_line_complex_data['normalized_monomer_count_ratio'].mean(),
    line_dash="dot",
    line_color="red"
)
fig.show()

In [10]:
# same thing but for essential monomers associated with complexes
fig = px.histogram(
    below_line_essential_complex_data,
    x='normalized_monomer_count_ratio',
    nbins=50,
    title='Distribution of normalized monomer count ratios for essential '
            'monomers below the line associated with complexes'
)
fig.add_vline(
    x=below_line_essential_complex_data['normalized_monomer_count_ratio'].mean(),
    line_dash="dot",
    line_color="red"
)
fig.show()

#### Since these are so low I think I should do with my list with 0.8 first and then see if I can try with a lower percentage later.

# Metabolism Process

In [11]:
# import kinetic reaction info from sim result
time = '600'
date = '2025-11-30'
experiment = 'output_objective_weights'
condition = 'basal'
entry = f'{experiment}_{time}_{date}'
folder = f'out/objective_weight/{condition}/{entry}/'

output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
output = output['agents']['0']
fba = output['listeners']['fba_results']
bulk = pd.DataFrame(output['bulk'])
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

In [12]:
# get commonly stored variables
metabolism = agent['ecoli-metabolism-redux-classic']
stoichiometry = metabolism.stoichiometry.copy()
reaction_names = metabolism.reaction_names
fba_new_reaction_ids = metabolism.parameters["fba_new_reaction_ids"]
fba_reaction_ids_to_base_reaction_ids = metabolism.parameters['fba_reaction_ids_to_base_reaction_ids']
metabolites = metabolism.metabolite_names.copy()
binary_kinetic_idx = metabolism.binary_kinetic_idx
exchange_molecules = metabolism.exchange_molecules
kinetic_enzymes = metabolism.parameters['kinetic_constraint_enzymes']
all_catalysts = metabolism.parameters["catalyst_ids"]
reaction_to_catalysts_dict = metabolism.parameters['reaction_catalysts']
catalysts_to_reactions_dict = {}
for rxn, cats in reaction_to_catalysts_dict.items():
    for cat in cats:
        if cat not in catalysts_to_reactions_dict:
            catalysts_to_reactions_dict[cat] = []
        catalysts_to_reactions_dict[cat].append(rxn)

S = stoichiometry .copy()
S = pd.DataFrame(S, index=metabolites , columns=reaction_names )
homeostatic_count = pd.DataFrame(fba["homeostatic_metabolite_counts"], columns=metabolism.homeostatic_metabolites).mean(axis=0)
homeostatic = pd.DataFrame(fba["target_homeostatic_dmdt"], columns=metabolism.homeostatic_metabolites).mean(axis=0)
maintenance = pd.DataFrame(fba["maintenance_target"][1:], columns=['maintenance_reaction']).mean(axis=0)
kinetic = pd.DataFrame(fba["target_kinetic_fluxes"], columns=metabolism.kinetic_constraint_reactions).mean(axis=0).copy()

kinetic_reaction_ids = metabolism.kinetic_constraint_reactions
allowed_exchange_uptake = metabolism.allowed_exchange_uptake
FREE_RXNS = ["TRANS-RXN-145", "TRANS-RXN0-545", "TRANS-RXN0-474",
             "ATPSYN-RXN (reverse)" # TODO: Riley added this one
             ]
# TODO: load in the above from script directly to avoid mismatches

In [13]:
# Get a list of below_line_monomer_ids that are in all_catalysts
below_line_monomer_catalyst_ids = [
    m for m in below_line_monomer_ids if m in all_catalysts]
below_line_essential_monomer_catalyst_ids = [
    m for m in below_line_essential_monomer_ids if m in all_catalysts]
below_line_complex_catalyst_ids = [
    c for c in below_line_complex_ids if c in all_catalysts]
below_line_essential_complex_catalyst_ids = [
    c for c in below_line_essential_complex_ids if c in all_catalysts]

print(f'Number of below line monomer catalysts: '
      f'{len(below_line_monomer_catalyst_ids)}')
print(f'Number of below line essential monomer catalysts: '
      f'{len(below_line_essential_monomer_catalyst_ids)}')
print(f'Number of below line complex catalysts: '
        f'{len(below_line_complex_catalyst_ids)}')
print(f'Number of below line essential complex catalysts: '
        f'{len(below_line_essential_complex_catalyst_ids)}')

Number of below line monomer catalysts: 654
Number of below line essential monomer catalysts: 72
Number of below line complex catalysts: 648
Number of below line essential complex catalysts: 140


In [14]:
# Map the catalyst lists to the reactions they catalyze
below_line_monomer_reaction_ids = []
for cat in below_line_monomer_catalyst_ids:
    if cat in catalysts_to_reactions_dict:
        below_line_monomer_reaction_ids.extend(
            catalysts_to_reactions_dict[cat])
below_line_monomer_reaction_ids = np.unique(below_line_monomer_reaction_ids).tolist()

below_line_essential_monomer_reaction_ids = []
for cat in below_line_essential_monomer_catalyst_ids:
    if cat in catalysts_to_reactions_dict:
        below_line_essential_monomer_reaction_ids.extend(
            catalysts_to_reactions_dict[cat])
below_line_essential_monomer_reaction_ids = np.unique(
    below_line_essential_monomer_reaction_ids).tolist()

below_line_complex_reaction_ids = []
for cat in below_line_complex_catalyst_ids:
    if cat in catalysts_to_reactions_dict:
        below_line_complex_reaction_ids.extend(
            catalysts_to_reactions_dict[cat])
below_line_complex_reaction_ids = np.unique(
    below_line_complex_reaction_ids).tolist()

below_line_essential_complex_reaction_ids = []
for cat in below_line_essential_complex_catalyst_ids:
    if cat in catalysts_to_reactions_dict:
        below_line_essential_complex_reaction_ids.extend(
            catalysts_to_reactions_dict[cat])
below_line_essential_complex_reaction_ids = np.unique(
    below_line_essential_complex_reaction_ids).tolist()
print(f'Number of below line monomer reaction ids: '
      f'{len(below_line_monomer_reaction_ids)}')
print(f'Number of below line essential monomer reaction ids: '
        f'{len(below_line_essential_monomer_reaction_ids)}')
print(f'Number of below line complex reaction ids: '
        f'{len(below_line_complex_reaction_ids)}')
print(f'Number of below line essential complex reaction ids: '
        f'{len(below_line_essential_complex_reaction_ids)}')


Number of below line monomer reaction ids: 1580
Number of below line essential monomer reaction ids: 163
Number of below line complex reaction ids: 6152
Number of below line essential complex reaction ids: 363


In [15]:
# Get one list of all reaxtions from combining
# below_line_monomer_reaction_ids and below_line_complex_reaction_ids
below_line_all_reaction_ids = np.unique(
    below_line_monomer_reaction_ids +
    below_line_complex_reaction_ids).tolist()
below_line_essential_reaction_ids = np.unique(
    below_line_essential_monomer_reaction_ids +
    below_line_essential_complex_reaction_ids).tolist()

print(f'Number of below line all reaction ids: '
      f'{len(below_line_all_reaction_ids)}')
print(f'Number of below line essential reaction ids: '
        f'{len(below_line_essential_reaction_ids)}')

Number of below line all reaction ids: 7475
Number of below line essential reaction ids: 519


## Test FBA

In [16]:
# All average reaction fluxes from short vEcoli sim
sim = pd.DataFrame(fba["estimated_fluxes"], columns= reaction_names).mean(axis=0).copy()

In [17]:
below_line_all_reaction_idx = np.array(
    [reaction_names.index(id) for id in below_line_all_reaction_ids
     if id in reaction_names])
below_line_essential_reaction_idx = np.array(
    [reaction_names.index(id) for id in below_line_essential_reaction_ids
        if id in reaction_names])

below_line_all_reaction_WT_fluxes = sim.iloc[below_line_all_reaction_idx]
below_line_essential_reaction_WT_fluxes = sim.iloc[
    below_line_essential_reaction_idx]

all_below_line_reaction_constraint_dict = dict(
    zip(below_line_all_reaction_idx, below_line_all_reaction_WT_fluxes))
all_below_line_essential_reaction_constraint_dict = dict(
    zip(below_line_essential_reaction_idx,
        below_line_essential_reaction_WT_fluxes))

# TODO: we also need to adjust the kinetic flux targets for some subset of
#  these reactions?


In [18]:
fba.keys()

dict_keys(['solution_fluxes', 'solution_dmdt', 'solution_residuals', 'time_per_step', 'estimated_fluxes', 'estimated_homeostatic_dmdt', 'homeostatic_metabolite_counts', 'target_homeostatic_dmdt', 'estimated_exchange_dmdt', 'estimated_intermediate_dmdt', 'target_kinetic_fluxes', 'target_kinetic_bounds', 'reaction_catalyst_counts', 'homeostatic_term', 'secretion_term', 'efficiency_term', 'kinetic_term', 'maintenance_target', 'loss_total', 'loss_kinetic', 'loss_homeostatic', 'loss_secretion', 'loss_efficiency', 'loss_diversity'])

In [19]:
# Create row from input data for comparison
input_row = {
    "overall_loss": np.array(fba['loss_total']).mean(),
    "kinetic_loss": np.array(fba['loss_kinetic']).mean(),
    "homeostatic_loss": np.array(fba['loss_homeostatic']).mean(),
    "overall_objective": None,
    "kinetic_counts_unweighted": None,
    "homeostatic_counts": None,
    "description": "Input data from short vEcoli sim"
}

In [20]:
# Get dm_dt for homeostatic metabolites from input data
homeostatic_metabolite_idx = np.array([metabolism.metabolite_names.index(m) for m in metabolism.homeostatic_metabolites])
# Convert fba["solution_dmdt"] to df
solution_dmdt_df = pd.DataFrame(fba["solution_dmdt"], columns=metabolism.metabolite_names)
# Get homeostatic metabolites only
dm_dt_homeostatic = solution_dmdt_df.iloc[:, homeostatic_metabolite_idx]
dm_dt_homeostatic_mean = dm_dt_homeostatic.mean(axis=0)
dm_dt_homeostatic_mean
# but this is in concentration not counts...

2-3-DIHYDROXYBENZOATE[c]    0.000029
2-KETOGLUTARATE[c]          0.000075
2-PG[c]                     0.000019
2K-4CH3-PENTANOATE[c]       0.000029
4-AMINO-BUTYRATE[c]         0.000064
                              ...   
NA+[p]                      0.000021
OXYGEN-MOLECULE[p]          0.000021
FE+3[p]                     0.000021
CA+2[p]                     0.000021
Pi[p]                       0.000021
Length: 172, dtype: float64

In [21]:
# Plot total loaa, kinetic loss, homeostatic loss over time
import plotly.express as px
loss_df = pd.DataFrame({
    "time": np.arange(len(fba['loss_total'])),
    "total_loss": fba['loss_total'],
    "kinetic_loss": fba['loss_kinetic'],
    "homeostatic_loss": fba['loss_homeostatic']
})
loss_df_melted = loss_df.melt(id_vars=['time'], value_vars=['total_loss', 'kinetic_loss', 'homeostatic_loss'], var_name='loss_type', value_name='loss_value')
fig = px.line(loss_df_melted, x='time', y='loss_value', color='loss_type', title='Losses over time')
fig.show()

In [22]:
def get_subset_S(S, met_of_interest):
    S_met = S.loc[met_of_interest, :]
    S_met = S_met.loc[:,~np.all(S_met == 0, axis=0)]
    return S_met, S_met.columns

def get_keys(dict, value):
    return [key for key in dict if np.any(np.isin(value, dict[key]))]

def test_NetworkFlowModel(
            objective_weights,
            kinetic_targets = pd.DataFrame(fba["target_kinetic_fluxes"],
                                           columns=metabolism.kinetic_constraint_reactions).loc[24, :].copy(),
            uptake_addition = set([]), uptake_removal = set([]), new_exchange_molecules = set([]),
            add_metabolite = None, add_reaction = None, add_kinetic = None, remove_reaction = None, force_reaction = None,
            add_homeostatic_demand = None,
            kinetic_constraint = None, kinetic_adjustment = None,
            solver_choice=cp.GLOP):
    # update exchanges
    uptake = metabolism.allowed_exchange_uptake.copy()
    uptake = set(uptake)
    uptake = uptake | uptake_addition
    uptake = uptake - uptake_removal

    exchange_molecules = metabolism.exchange_molecules.copy()
    exchange_molecules = exchange_molecules | new_exchange_molecules

    # update stoichiometry
    reaction_names = metabolism.reaction_names.copy()
    kinetic_reaction_ids = metabolism.kinetic_constraint_reactions.copy()
    kinetic = kinetic_targets
    metabolites = metabolism.metabolite_names.copy()
    homeostatic_counts = homeostatic_count.copy() * metabolism.counts_to_molar.asNumber()

    S_new = stoichiometry.copy()

    if add_metabolite is not None: # add to metabolites list because they are currently not included in the model
        for m in add_metabolite:
            if m not in metabolites:
                metabolites.append(m)
        # append rows of zeros to S_new of length add_metabolite
        S_new = np.concatenate((S_new, np.zeros((len(add_metabolite), S_new.shape[1]))), axis=0)

    if add_reaction is not None:
        # assert add_reaction is a dictionary
        assert isinstance(add_reaction, dict)

        for r,s in add_reaction.items():
            if r not in reaction_names:
                reaction_names.append(r)
            # append columns of reaction stoich to S_new of length add_reaction
            new_reaction = np.zeros((S_new.shape[0], 1))
            for m, v in s.items():
                new_reaction[metabolites.index(m), 0] = v
            S_new = np.concatenate((S_new, new_reaction), axis=1)

    if add_kinetic is not None:
        # assert add_kinetic is a dictionary
        assert isinstance(add_kinetic, dict)

        for r, v in add_kinetic.items():
            if r not in kinetic_reaction_ids:
                kinetic_reaction_ids.append(r)
                kinetic[r] = v
            if r in kinetic_reaction_ids:
                kinetic[r] = v

    if remove_reaction is not None:
        for r in remove_reaction:
            r_idx = reaction_names.index(r)
            S_new = np.delete(S_new, r_idx, axis=1)
            reaction_names.remove(r)
            if r in kinetic_reaction_ids:
                kinetic_reaction_ids.remove(r)
                del kinetic[r]

    if force_reaction is not None:
        force_reaction_idx = np.array([reaction_names.index(r) for r in force_reaction])
    else:
        force_reaction_idx = force_reaction

    if add_homeostatic_demand is not None:
        # assert add_homeostatic_demand is a set
        assert isinstance(add_homeostatic_demand, list)

        for met in add_homeostatic_demand:
            homeostatic[met] = 100
            homeostatic_counts[met] = 1

    # Solve NetworkFlowModel
    model = NetworkFlowModel(
            stoich_arr=S_new,
            metabolites=metabolites,
            reactions=reaction_names,
            homeostatic_metabolites=metabolism.homeostatic_metabolites,
            kinetic_reactions=kinetic_reaction_ids,
            free_reactions=FREE_RXNS)
    model.set_up_exchanges(exchanges=exchange_molecules, uptakes=uptake)
    solution: FlowResult = model.solve(
            homeostatic_concs=homeostatic_counts, # in conc
            homeostatic_dm_targets=np.array(list(dict(homeostatic).values())), # *10^7
            maintenance_target=maintenance, # *10^6 ish
            kinetic_targets=np.array(list(dict(kinetic).values())), # *10^6 ish
            # binary_kinetic_idx=binary_kinetic_idx, #7646
            binary_kinetic_idx=None,
            force_flow_idx=force_reaction_idx,
            objective_weights=objective_weights, #same
            upper_flux_bound= 1000000000, # increase to 10^9 because notebook runs FlowResult using Counts, WC runs using conc.,
            kinetic_constraint=kinetic_constraint,
            kinetic_adjustment=kinetic_adjustment,
            solver=solver_choice) #SCS. ECOS, MOSEK
    return solution

In [23]:
obj_by_perturbation = {}

In [24]:
# Normal FBA
objective_weights = {'secretion': 0.01, 'efficiency': 1e-6, 'kinetics': 1e-5, 'homeostatic': 1}
normal_solution = (
    test_NetworkFlowModel(
        objective_weights))
loss_overall = normal_solution.loss_total
loss_kinetic = normal_solution.loss_kinetic
loss_homeostatic = normal_solution.loss_homeostatic
oofv = normal_solution.objective
kinetic_soln = normal_solution.kinetic_term
homeostatic_soln = normal_solution.homeostatic_term
obj_by_perturbation['normal_fba'] = {
    "overall_loss": loss_overall,
    "kinetic_loss": loss_kinetic,
    "homeostatic_loss": loss_homeostatic,
    "overall_objective": oofv,
    "kinetic_counts_unweighted": kinetic_soln,
    "homeostatic_counts": homeostatic_soln,
    "description": "Normal FBA"}

In [25]:
# FBA with 80% kinetic constraint on all below line reactions
objective_weights = {'secretion': 0.01, 'efficiency': 1e-6, 'kinetics': 1e-5, 'homeostatic': 1}
solution_80pct_all_below_line = (
    test_NetworkFlowModel(
        objective_weights,
        kinetic_constraint=all_below_line_reaction_constraint_dict,
        kinetic_adjustment=0.8
    ))
loss_overall = solution_80pct_all_below_line.loss_total
loss_kinetic = solution_80pct_all_below_line.loss_kinetic
loss_homeostatic = solution_80pct_all_below_line.loss_homeostatic
oofv = solution_80pct_all_below_line.objective
kinetic_soln = solution_80pct_all_below_line.kinetic_term
homeostatic_soln = solution_80pct_all_below_line.homeostatic_term
obj_by_perturbation['fba_80pct_all_below_line'] = {
    "overall_loss": loss_overall,
    "kinetic_loss": loss_kinetic,
    "homeostatic_loss": loss_homeostatic,
    "overall_objective": oofv,
    "kinetic_counts_unweighted": kinetic_soln,
    "homeostatic_counts": homeostatic_soln,
    "description": "FBA with 80% kinetic constraint on all below line "
                   "reactions"}

In [26]:
# FBA with 50% kinetic constraint on all below line reactions
objective_weights = {'secretion': 0.01, 'efficiency': 1e-6, 'kinetics': 1e-5, 'homeostatic': 1}
solution_50pct_all_below_line = (
    test_NetworkFlowModel(
        objective_weights,
        kinetic_constraint=all_below_line_reaction_constraint_dict,
        kinetic_adjustment=0.5
    ))
loss_overall = solution_50pct_all_below_line.loss_total
loss_kinetic = solution_50pct_all_below_line.loss_kinetic
loss_homeostatic = solution_50pct_all_below_line.loss_homeostatic
oofv = solution_50pct_all_below_line.objective
kinetic_soln = solution_50pct_all_below_line.kinetic_term
homeostatic_soln = solution_50pct_all_below_line.homeostatic_term
obj_by_perturbation['fba_50pct_all_below_line'] = {
    "overall_loss": loss_overall,
    "kinetic_loss": loss_kinetic,
    "homeostatic_loss": loss_homeostatic,
    "overall_objective": oofv,
    "kinetic_counts_unweighted": kinetic_soln,
    "homeostatic_counts": homeostatic_soln,
    "description": "FBA with 50% kinetic constraint on all below "
                   "line reactions"}


In [27]:
# FBA with 80% kinetic constraint on all essential below line reactions
objective_weights = {'secretion': 0.01, 'efficiency': 1e-6, 'kinetics': 1e-5, 'homeostatic': 1}
solution_80pct_all_below_line_essential = (
    test_NetworkFlowModel(
        objective_weights,
        kinetic_constraint=all_below_line_essential_reaction_constraint_dict,
        kinetic_adjustment=0.8,
    ))
loss_overall = solution_80pct_all_below_line_essential.loss_total
loss_kinetic = solution_80pct_all_below_line_essential.loss_kinetic
loss_homeostatic = solution_80pct_all_below_line_essential.loss_homeostatic
oofv = solution_80pct_all_below_line_essential.objective
kinetic_soln = solution_80pct_all_below_line_essential.kinetic_term
homeostatic_soln = solution_80pct_all_below_line_essential.homeostatic_term
obj_by_perturbation['fba_80pct_all_below_line_essential'] = {
    "overall_loss": loss_overall,
    "kinetic_loss": loss_kinetic,
    "homeostatic_loss": loss_homeostatic,
    "overall_objective": oofv,
    "kinetic_counts_unweighted": kinetic_soln,
    "homeostatic_counts": homeostatic_soln,
    "description": "FBA with 80% kinetic constraint on all essential "
                   "below line reactions"}


In [28]:
# FBA with 50% kinetic constraint on all essential below line reactions
objective_weights = {'secretion': 0.01, 'efficiency': 1e-6, 'kinetics': 1e-5, 'homeostatic': 1}
solution_50pct_all_below_line_essential = (
    test_NetworkFlowModel(
        objective_weights,
        kinetic_constraint=all_below_line_essential_reaction_constraint_dict,
        kinetic_adjustment=0.5,
    ))
loss_overall = solution_50pct_all_below_line_essential.loss_total
loss_kinetic = solution_50pct_all_below_line_essential.loss_kinetic
loss_homeostatic = solution_50pct_all_below_line_essential.loss_homeostatic
oofv = solution_50pct_all_below_line_essential.objective
kinetic_soln = solution_50pct_all_below_line_essential.kinetic_term
homeostatic_soln = solution_50pct_all_below_line_essential.homeostatic_term
obj_by_perturbation['fba_50pct_all_below_line_essential'] = {
    "overall_loss": loss_overall,
    "kinetic_loss": loss_kinetic,
    "homeostatic_loss": loss_homeostatic,
    "overall_objective": oofv,
    "kinetic_counts_unweighted": kinetic_soln,
    "homeostatic_counts": homeostatic_soln,
    "description": "FBA with 50% kinetic constraint on all essential "
                   "below line reactions"}

In [29]:
# Turn obj_by_perturbation into a df
obj_by_perturbation_df = pd.DataFrame(obj_by_perturbation).T
# Multiply _loss columns by 1e-6 to convert from counts to concentrations
loss_cols = [col for col in obj_by_perturbation_df.columns if 'loss' in col]
obj_by_perturbation_df[loss_cols] = obj_by_perturbation_df[loss_cols] * 1e-6

# Add column as the first row for the input data
input_row_df = pd.DataFrame([input_row], index=['input_data'])
obj_by_perturbation_df = pd.concat([input_row_df, obj_by_perturbation_df], axis=0)

obj_by_perturbation_df

,overall_loss,kinetic_loss,homeostatic_loss,overall_objective,kinetic_counts_unweighted,homeostatic_counts,description
input_data,0.238814,0.000262,0.223144,None,None,None,Input data from short vEcoli sim
normal_fba,0.165772,0.000162,0.155495,165772.210767,16193542.608446,155494.734187,Normal FBA
fba_80pct_all_below_line,0.429704,0.000206,0.421359,429704.057108,20592606.795457,421359.021008,FBA with 80% kinetic constraint on all below l...
fba_50pct_all_below_line,3.905545,0.000311,3.894702,3905545.04448,31097387.483056,3894701.88162,FBA with 50% kinetic constraint on all below l...
fba_80pct_all_below_line_essential,0.276788,0.000161,0.267233,276788.333666,16091926.983657,267233.053764,FBA with 80% kinetic constraint on all essenti...
fba_50pct_all_below_line_essential,0.451266,0.000161,0.442031,451265.791482,16097811.84144,442030.588469,FBA with 50% kinetic constraint on all essenti...


#### It is really interesting that in the above constraining all reactions from 80 to 50 percent actually decreases the overall loss, but this was not the case with kinetic or essential reactions only.